In [ ]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('/content/train_data1.csv')

# Check data types and nulls
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16209 entries, 0 to 16208
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             16209 non-null  int64  
 1   date           16209 non-null  object 
 2   price          16209 non-null  int64  
 3   bedrooms       16209 non-null  int64  
 4   bathrooms      16209 non-null  float64
 5   sqft_living    16209 non-null  int64  
 6   sqft_lot       16209 non-null  int64  
 7   floors         16209 non-null  float64
 8   waterfront     16209 non-null  int64  
 9   view           16209 non-null  int64  
 10  condition      16209 non-null  int64  
 11  grade          16209 non-null  int64  
 12  sqft_above     16209 non-null  int64  
 13  sqft_basement  16209 non-null  int64  
 14  yr_built       16209 non-null  int64  
 15  yr_renovated   16209 non-null  int64  
 16  zipcode        16209 non-null  int64  
 17  lat            16209 non-null  float64
 18  long  

In [ ]:
import pandas as pd

# --- STEP 0: Original Shape ---
print(f"Original Shape: {df.shape}")

# --- STEP 1: DROP IRRELEVANT COLUMNS ---
df = df.drop(['id'], axis=1)

# --- STEP 2: HANDLE DATES ---
df['date'] = pd.to_datetime(df['date'])
df['sale_year'] = df['date'].dt.year
df['sale_month'] = df['date'].dt.month
df = df.drop(['date'], axis=1)

# --- STEP 3: FEATURE ENGINEERING (AGE & RENOVATION) ---
df['house_age'] = df['sale_year'] - df['yr_built']
df['is_renovated'] = (df['yr_renovated'] > 0).astype(int)

# --- STEP 4: OUTLIER REMOVAL ---
df = df[df['bedrooms'] < 15]

# --- STEP 5: CATEGORICAL HANDLING ---
df['zipcode'] = df['zipcode'].astype('category')

# --- FINAL STATUS ---
print("-" * 30)
print(f"Cleaned Data Shape: {df.shape}")
print(df.head())


Original Shape: (16209, 21)
------------------------------
Cleaned Data Shape: (16208, 23)
    price  bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  \
0  268643         4       2.25         1810      9240     2.0           0   
1  245000         3       2.50         1600      2788     2.0           0   
2  200000         4       2.50         1720      8638     2.0           0   
3  352499         2       2.25         1240       705     2.0           0   
4  232000         3       2.00         1280     13356     1.0           0   

   view  condition  grade  ...  yr_renovated  zipcode      lat     long  \
0     0          3      7  ...             0    98055  47.4362 -122.187   
1     0          4      7  ...             0    98031  47.4034 -122.187   
2     0          3      8  ...             0    98003  47.2704 -122.313   
3     0          3      7  ...             0    98027  47.5321 -122.073   
4     0          3      7  ...             0    98042  47.3715 -122.074

Connect to google drive

In [8]:
from google.colab import drive
import os

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Create folder to save images
SAVE_FOLDER = "/content/drive/MyDrive/sat_img_train_data"
os.makedirs(SAVE_FOLDER, exist_ok=True)

print(f" Target Folder Ready: {SAVE_FOLDER}")


Mounted at /content/drive
 Target Folder Ready: /content/drive/MyDrive/sat_img_train_data


Parameters to download images from mapbox

In [10]:
MAPBOX_TOKEN = "pk.eyJ1IjoiYm5zcGF2YW5rdW1hciIsImEiOiJjbWpwbGg3ZnIzdHE2M2hxeDd3c3g0NWtjIn0.rML8whbRvPCU_gURm8SLXw"

# Map settings
STYLE = "mapbox/satellite-v9"   # Map style
ZOOM = 18                        # Zoom level (16–19 is good for houses)
SIZE = "512x512"                 # Max allowed
REQUEST_DELAY = 0.4              # Delay between requests (seconds)
MAX_RETRIES = 5                  # Retry attempts per image

print(" Mapbox configuration set")


 Mapbox configuration set


Download img from mapbox using api token

In [ ]:
import requests
import time
from tqdm import tqdm
import os

def download_images_safe(df):
    print(f"Starting download for {len(df)} houses...")
    print(f"Saving to: {SAVE_FOLDER}")

    for idx, row in tqdm(df.iterrows(), total=len(df)):
        filename = f"house_{idx}.jpg"
        filepath = os.path.join(SAVE_FOLDER, filename)

        # Skip if already downloaded
        if os.path.exists(filepath):
            continue

        # Get coordinates
        try:
            lat = float(row["lat"])
            lon = float(row["long"])
        except Exception:
            print(f"Invalid lat/long at index {idx}")
            continue

        # Construct Mapbox URL
        url = (
            f"https://api.mapbox.com/styles/v1/{STYLE}/static/"
            f"{lon},{lat},{ZOOM},0/{SIZE}"
            f"?access_token={MAPBOX_TOKEN}"
        )

        # Retry logic
        for attempt in range(MAX_RETRIES):
            try:
                r = requests.get(url, timeout=15)
                if r.status_code == 200:
                    with open(filepath, "wb") as f:
                        f.write(r.content)
                    break
                elif r.status_code == 429:
                    print(" Rate limit hit → sleeping 60s")
                    time.sleep(60)
                else:
                    print(f"HTTP {r.status_code} at index {idx}")
                    break
            except Exception as e:
                print(f"Network error at index {idx}: {e}")
                time.sleep(3)

        # Rate-limit delay
        time.sleep(REQUEST_DELAY)

    print("\n Download complete!")

# --- RUN THE DOWNLOAD ---
download_images_safe(df)

Starting download for 16208 houses...
Saving to: /content/drive/MyDrive/sat_img_train_data


100%|██████████| 16208/16208 [3:35:01<00:00,  1.26it/s]


 Download complete!


In [ ]:
import os
import pandas as pd

# 1. SETUP
# Make sure your dataframe 'df' is loaded
if 'df' not in locals():
    df = pd.read_csv('/content/train_data1.csv')

# 2. COUNT FILES
folder_path = "/content/drive/MyDrive/sat_img_train_data"
files = os.listdir(folder_path)
image_count = len([f for f in files if f.endswith('.jpg')])
csv_count = len(df)

# 3. REPORT
print(f"📊 CSV Rows:      {csv_count}")
print(f"📂 Images Found:  {image_count}")

if image_count == csv_count:
    print("\n✅ PERFECT MATCH! You are ready to train.")
else:
    missing = csv_count - image_count
    print(f"\n⚠️ WARNING: You are missing {missing} images.")
    print("Don't worry! Just run the 'Download Script' again.")
    print("It will automatically skip the ones you have and fetch the missing ones.")

📊 CSV Rows:      16208
📂 Images Found:  16208

✅ PERFECT MATCH! You are ready to train.


In [ ]:
import os
import pandas as pd

# 1. SETUP: Ensure DataFrame is loaded
# (Assuming 'df' is your variable)
if 'df' not in locals():
    df = pd.read_csv('/content/train_data1.csv')

# 2. CREATE A FILENAME COLUMN
# This maps the *actual* index to the filename, handling gaps safely.
# If index 3193 is missing, this code just skips it naturally.
df['filename'] = "house_" + df.index.astype(str) + ".jpg"

# 3. VERIFY FILES EXIST
# This checks the folder for every single row in your CSV.
folder_path = "/content/drive/MyDrive/sat_img_train_data"

def image_exists(filename):
    full_path = os.path.join(folder_path, filename)
    return os.path.exists(full_path)

# Filter: Keep only rows where the image actually exists
initial_count = len(df)
df_clean = df[df['filename'].apply(image_exists)].copy()
final_count = len(df_clean)

# 4. REPORT
print(f"📊 Original Rows: {initial_count}")
print(f"✅ Valid Images:  {final_count}")
print(f"🗑️ Removed Rows:  {initial_count - final_count}")

if initial_count != final_count:
    print("⚠️ Cleaned up some missing files. Use 'df_clean' for training now!")
else:
    print("🎉 Dataset is perfectly synced!")

📊 Original Rows: 16208
✅ Valid Images:  16208
🗑️ Removed Rows:  0
🎉 Dataset is perfectly synced!


Similarly download the img dataset for the test data

In [6]:
import pandas as pd
import numpy as np

# Load the dataset
df_test = pd.read_csv('/content/test2_data1.csv')


# Check data types and nulls
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5404 entries, 0 to 5403
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             5404 non-null   int64  
 1   date           5404 non-null   object 
 2   bedrooms       5404 non-null   int64  
 3   bathrooms      5404 non-null   float64
 4   sqft_living    5404 non-null   int64  
 5   sqft_lot       5404 non-null   int64  
 6   floors         5404 non-null   float64
 7   waterfront     5404 non-null   int64  
 8   view           5404 non-null   int64  
 9   condition      5404 non-null   int64  
 10  grade          5404 non-null   int64  
 11  sqft_above     5404 non-null   int64  
 12  sqft_basement  5404 non-null   int64  
 13  yr_built       5404 non-null   int64  
 14  yr_renovated   5404 non-null   int64  
 15  zipcode        5404 non-null   int64  
 16  lat            5404 non-null   float64
 17  long           5404 non-null   float64
 18  sqft_liv

In [9]:



TEST_SAVE_FOLDER = "/content/drive/MyDrive/sat_img_test_data"
os.makedirs(TEST_SAVE_FOLDER, exist_ok=True)


In [11]:
import requests
import time
from tqdm import tqdm

def download_test_images(dataframe, output_folder):
    print(f"Starting download for {len(dataframe)} test houses...")
    print(f"Saving to: {output_folder}")

    for idx, row in tqdm(dataframe.iterrows(), total=len(dataframe)):
        # We use 'test_house_' prefix to differentiate files clearly
        filename = f"test_house_{idx}.jpg"
        filepath = os.path.join(output_folder, filename)

        # Skip if already downloaded
        if os.path.exists(filepath):
            continue

        # Get coordinates (Ensure your CSV has 'lat' and 'long' columns)
        try:
            lat = float(row["lat"])
            lon = float(row["long"])
        except Exception:
            print(f"Invalid lat/long at index {idx}")
            continue

        # Construct Mapbox URL
        url = (
            f"https://api.mapbox.com/styles/v1/{STYLE}/static/"
            f"{lon},{lat},{ZOOM},0/{SIZE}"
            f"?access_token={MAPBOX_TOKEN}"
        )

        # Retry logic
        for attempt in range(MAX_RETRIES):
            try:
                r = requests.get(url, timeout=15)
                if r.status_code == 200:
                    with open(filepath, "wb") as f:
                        f.write(r.content)
                    break # Success!
                elif r.status_code == 429:
                    print("Rate limit hit → sleeping 60s")
                    time.sleep(60)
                else:
                    print(f"HTTP {r.status_code} at index {idx}")
                    break # Fatal error for this image
            except Exception as e:
                print(f"Network error at index {idx}: {e}")
                time.sleep(3)

        # Rate-limit delay
        time.sleep(REQUEST_DELAY)

    print("\Download complete!")

# --- RUN THE DOWNLOAD ON TEST DATA ---
download_test_images(df_test, TEST_SAVE_FOLDER)

<>:54: SyntaxWarning: invalid escape sequence '\D'
<>:54: SyntaxWarning: invalid escape sequence '\D'
/tmp/ipython-input-2255326875.py:54: SyntaxWarning: invalid escape sequence '\D'
  print("\Download complete!")


Starting download for 5404 test houses...
Saving to: /content/drive/MyDrive/sat_img_test_data


100%|██████████| 5404/5404 [1:18:12<00:00,  1.15it/s]

\Download complete!
